In [1]:
import tensorflow as tf
import xgboost as xgb
import numpy as np
import pandas as pd
import datetime
import os
from sqlalchemy import Column, String, create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [2]:
PATH = '/data/share/china_stocks/data/main_intervals/future_interval/IC_15min.csv'

In [3]:
col = ["id","interval","time","open_mid","close_mid","high","low","volume","vwap","spread","adjust","type"]

In [4]:
df = pd.read_csv(PATH,header=None)
df.columns = col

In [5]:
df["time"] = pd.to_datetime(df["time"])

In [6]:
p1 = df["time"].dt.hour == 9
p2 = df["time"].dt.minute == 30

In [7]:
df = df[p1 & p2].copy().reset_index(drop = True)

In [8]:
df["open_mid_ad"] = df["open_mid"] - df["adjust"] 

In [9]:
df["open_mid_t+1"] = df["open_mid"].shift(-1)
df["open_mid_t+2"] = df["open_mid_ad"].shift(-2)

In [10]:
df["future_return500"] = df["open_mid_t+2"]/df["open_mid_t+1"] -1

In [11]:
del df["id"]
del df["interval"]
del df["volume"]

In [12]:
df.loc[15:]

,time,open_mid,close_mid,high,low,vwap,spread,adjust,type,open_mid_ad,open_mid_t+1,open_mid_t+2,future_return500
15,2015-05-12 09:30:00,8499.6,8497.5,8525.3,8493.5,8509.809666,0.000129,0.0,0,8499.6,8596.3,8654.2,0.006735
16,2015-05-13 09:30:00,8596.3,8523.2,8612.7,8494.8,8547.617078,0.000119,0.0,0,8596.3,8530.0,8610.0,0.009379
17,2015-05-14 09:30:00,8530.0,8599.9,8612.3,8530.0,8573.029822,0.000164,-124.2,0,8654.2,8610.0,8640.3,0.003519
18,2015-05-15 09:30:00,8610.0,8553.6,8610.0,8545.9,8569.955773,0.000127,0.0,0,8610.0,8640.3,8805.7,0.019143
19,2015-05-18 09:30:00,8640.3,8610.9,8656.6,8580.1,8611.809269,0.000126,0.0,0,8640.3,8805.7,9121.5,0.035863
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,2019-10-30 09:30:00,4963.2,4963.2,4963.2,4963.2,4963.200000,0.000081,0.0,0,4963.2,4928.5,4884.1,-0.009009
1106,2019-10-31 09:30:00,4928.5,4928.5,4928.5,4928.5,4928.500000,0.000041,0.0,0,4928.5,4884.1,4966.2,0.016810
1107,2019-11-01 09:30:00,4884.1,4882.0,4884.1,4882.0,4882.962500,0.000300,0.0,0,4884.1,4966.2,4970.5,0.000866
1108,2019-11-04 09:30:00,4966.2,4966.2,4966.2,4966.2,4966.200000,0.000081,0.0,0,4966.2,4970.5,NaN,NaN


In [13]:
df = df[["future_return500","time"]]

In [14]:
df["time"] = df["time"].apply(lambda x:str(x)[:10])

In [15]:
df.columns = ["future500_return","date_time"]

In [16]:
df.to_csv("future500_open.csv")

In [17]:
df

,future500_return,date_time
0,0.066260,2015-04-20
1,0.004401,2015-04-21
2,0.006689,2015-04-22
3,0.014574,2015-04-23
4,-0.007337,2015-04-24
...,...,...
1105,-0.009009,2019-10-30
1106,0.016810,2019-10-31
1107,0.000866,2019-11-01
1108,NaN,2019-11-04
